In [1]:
%%pyspark
df = spark.read.load('abfss://files@datalake4awmfpe.dfs.core.windows.net/products/products.csv', format='csv'
, header=True
)
display(df.limit(10))

StatementMeta(spark4awmfpe, 0, 2, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7b6095c7-ef2e-4b9d-aaec-b3c93531a727)

In [2]:
delta_table_path = "/delta/products-delta"
df.write.format("delta").save(delta_table_path)

StatementMeta(spark4awmfpe, 0, 3, Finished, Available, Finished)

In [3]:
from delta.tables import *
from pyspark.sql.functions import *


deltaTable = DeltaTable.forPath(spark, delta_table_path)


deltaTable.update(
    condition = "ProductID == 771",
    set = { "ListPrice": "ListPrice * 0.9" })


deltaTable.toDF().show(10)


StatementMeta(spark4awmfpe, 0, 4, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [5]:
new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
new_df.show(10)

StatementMeta(spark4awmfpe, 0, 6, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [6]:
deltaTable.history(10).show(20, False, True)

StatementMeta(spark4awmfpe, 0, 7, Finished, Available, Finished)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 1                                                                                                                                                                                                                                     
 timestamp           | 2025-04-09 21:13:01.234                                                                                                                                                                                                               
 userId              | null                                                                                                                                                                                                                   

In [7]:
spark.sql("CREATE DATABASE AdventureWorks")
spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)


StatementMeta(spark4awmfpe, 0, 8, Finished, Available, Finished)

+----------------------------+-----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                              |comment|
+----------------------------+-----------------------------------------------------------------------+-------+
|ProductID                   |string                                                                 |null   |
|ProductName                 |string                                                                 |null   |
|Category                    |string                                                                 |null   |
|ListPrice                   |string                                                                 |null   |
|                            |                                                                       |       |
|# Detailed Table Information|                                                                       |       |
|

In [8]:
%%sql

USE AdventureWorks;

SELECT * FROM ProductsExternal;


StatementMeta(spark4awmfpe, 0, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [9]:
df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)


StatementMeta(spark4awmfpe, 0, 11, Finished, Available, Finished)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                       |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                          |null   |
|ProductName                 |string                                                                                                                          |null   |
|Category                    |string                                                                                                                          |n

In [10]:
%%sql

USE AdventureWorks;

SELECT * FROM ProductsManaged;


StatementMeta(spark4awmfpe, 0, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [12]:
%%sql

USE AdventureWorks;

DROP TABLE IF EXISTS ProductsExternal;
DROP TABLE IF EXISTS ProductsManaged;


StatementMeta(spark4awmfpe, 0, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
%%sql

USE AdventureWorks;

CREATE TABLE Products
USING DELTA
LOCATION '/delta/products-delta';


StatementMeta(spark4awmfpe, 0, 20, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
%%sql

USE AdventureWorks;

SELECT * FROM Products;


StatementMeta(spark4awmfpe, 0, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [15]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *


inputPath = '/data/'
mssparkutils.fs.mkdirs(inputPath)


jsonSchema = StructType([
    StructField("device", StringType(), False),
    StructField("status", StringType(), False)
])


iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)


device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''
mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
print("Source stream created...")


StatementMeta(spark4awmfpe, 0, 23, Finished, Available, Finished)

Source stream created...


In [16]:
delta_stream_table_path = '/delta/iotdevicedata'
checkpointpath = '/delta/checkpoint'

deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(spark4awmfpe, 0, 24, Finished, Available, Finished)

Streaming to delta sink...


In [17]:
df = spark.read.format("delta").load(delta_stream_table_path)
display(df)


StatementMeta(spark4awmfpe, 0, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 400c4cea-cbb4-4331-bcde-df87ddc76566)

In [18]:
spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))


StatementMeta(spark4awmfpe, 0, 26, Finished, Available, Finished)

DataFrame[]

In [19]:
%%sql

SELECT * FROM IotDeviceData;


StatementMeta(spark4awmfpe, 0, 27, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [20]:
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)


StatementMeta(spark4awmfpe, 0, 28, Finished, Available, Finished)

True

In [21]:
%%sql

SELECT * FROM IotDeviceData;


StatementMeta(spark4awmfpe, 0, 29, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [22]:
deltastream.stop()


StatementMeta(spark4awmfpe, 0, 30, Finished, Available, Finished)